Homework 4: Neural Language Models (& 🎃 SpOoKy 👻 authors 🧟 data) - Task 2
----

### Names
----
Names: __YOUR NAMES HERE__ (Write these in every notebook you submit.)

Task 2: Training your own word embeddings (15 points)
--------------------------------

For this task, you'll use the `gensim` package to train your own embeddings for both words and characters. These will eventually act as inputs to your neural language model.

In [1]:
# here are several dependencies to install
# !python --version
# !python -m pip install --upgrade pip setuptools wheel
# !pip install nltk
# !pip install gensim
# !pip install torch torchvision torchinfo

In [2]:
# import your libraries here

# Remember to restart your kernel if you change the contents of this file!
import neurallm_utils as nutils

# for word embeddings
# if not installed, run the following command:
# !pip install gensim
from gensim.models import Word2Vec

import torch
import torch.nn as nn

ModuleNotFoundError: No module named 'neurallm_utils'

In [ ]:
# If running on google colab, you'll need to mount your drive to access data files

# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
# constants you may find helpful. Edit as you would like.

# The dimensions of word embedding. 
# This variable will be used throughout the program
# DO NOT WRITE "50" WHEN YOU ARE REFERRING TO THE EMBEDDING SIZE
EMBEDDINGS_SIZE = 50

EMBEDDING_SAVE_FILE_WORD = f"spooky_embedding_word_{EMBEDDINGS_SIZE}.model" # The file to save your word embeddings to
EMBEDDING_SAVE_FILE_CHAR = f"spooky_embedding_char_{EMBEDDINGS_SIZE}.model" # The file to save your char embeddings to
TRAIN_FILE = 'spooky_author_train.csv' # The file to train your language model on


Train embeddings on provided dataset
---

In [ ]:
# your code here
# use the provided utility functions to read in the data


data = [['this', 'is', 'the', 'first', 'sentence', 'for', 'word2vec'],
			['this', 'is', 'the', 'second', 'sentence'],
			['yet', 'another', 'sentence'],
			['one', 'more', 'sentence'],
			['and', 'the', 'final', 'sentence']]


# read the spooky data in both by character and by word using the read_file_spooky function in the 
# provided utils


# print out the first two sentences in each format
# make sure we can read the output easily without scrolling to the side too much


8. What character represents spaces when we tokenize by character? __YOUR ANSWER HERE__
9. Read the word2vec documentation. What do the following parameters signify?
    - embeddings_size: __YOUR ANSWER HERE__
    - window: __YOUR ANSWER HERE__
    - min_count: __YOUR ANSWER HERE__
    - sg: __YOUR ANSWER HERE__

In [ ]:
# 10 points
# create your word embeddings
# use the skip gram algorithm and a window size of 5
# min_count should be 1
# takes ~3.3 sec on Felix's computer for character embeddings using skip-gram with window size 5
# takes ~3.3 sec on Felix's computer for word embeddings using skip-gram with window size 5 


def train_word2vec(data: list[list[str]], embeddings_size: int,
                    window: int = 5, min_count: int = 1, sg: int = 1) -> Word2Vec:
    """
    Create new word embeddings based on our data.

    Params:
        data: The corpus
        embeddings_size: The dimensions in each embedding

    Returns:
        A gensim Word2Vec model
        https://radimrehurek.com/gensim/models/word2vec.html
    """

    
    pass


# After you are happy with this function, copy + paste it into the bottom of 
# your neurallm_utils.py file
# You'll need it for the next task!
def create_embedder(raw_embeddings: Word2Vec) -> torch.nn.Embedding:
    """
    Create a PyTorch embedding layer based on our data.

    We will *first* train a Word2Vec model on our data.
    Then, we'll use these weights to create a PyTorch embedding layer.
        `nn.Embedding.from_pretrained(weights)`


    PyTorch docs: https://pytorch.org/docs/stable/generated/torch.nn.Embedding.html#torch.nn.Embedding.from_pretrained
    Gensim Word2Vec docs: https://radimrehurek.com/gensim/models/word2vec.html

    Pay particular attention to the *types* of the weights and the types required by PyTorch.

    Params:
        data: The corpus
        embeddings_size: The dimensions in each embedding

    Returns:
        A PyTorch embedding layer
    """

    # Hint:
    # For later tasks, we'll need two mappings: One from token to index, and one from index to tokens.
    # It might be a good idea to store these as properties of your embedder.
    # e.g. `embedder.token_to_index = ...`
    
    pass

In [ ]:

# Create and save both sets (word and character based) of Word2Vec embeddings. 
# Use the provided utility functions in nutils.
# These will be (re)loaded in the next notebook.


In [ ]:
# load them in again to make sure that this works and is still fast

In [ ]:
# now create the embedders


In [ ]:
# take a look at your saved token to index and index to token mappings in your embedders to make sure they make sense
# AND that they are both dictionaries mapping from int to str or vice versa!
# don't leave a ton of output in your notebook when you turn it in, but you need to understand this,
# and it's an easy place to make a mistake that's hard to debug later.
# do leave whatever code you use here, comment it out if it produces a lot of output

In [ ]:
# 4 points
# print out the vocabulary size for your embeddings for both your word
# embeddings and your character embeddings
# label which is which when you print them out
